Universidad del valle de Guatemala</br>
Facultad de Ingeniería</br>
Departamento de ciencias de la Computación</br>
Inteligencia Artificial

# Laboratorio 10: Clasificador de Imágenes (Perros vs Gatos) </br>
Deberá construir un modelo basado en redes neuronales que le permita clasificar perros y gatos basado en un input
de imágen. Para esto, deberá usar técnicas basadas en Deep Learning relacionadas con el manejo de imágenes
como las redes convolucionales. 

Grupo # 4</br>
Gabriel Vicente 2049

### Task 1.1 - Lectura del Dataset

En esta ocasión no recibirá un documento único como entrada de datos, por el contrario deberá recibir un conjunto
de imágenes que representan perros y gatos. Estos datos los puede descargar de este dataset de Kaggle. Deberá
leer estas imágenes para crear un dataset en memoria que haga referencia a la imagen con sus características y a
la categoría a la que pertenece. Recuerde que debe dividir su dataset según el uso y como se mencionó
anteriormente. Además, considere que la cantidad de imágenes de perros y gatos puede estar desigual por lo que
deberá aplicar las técnicas aprendidas durante el curso para lidiar con este problema Son libres de usar todas las
imágenes del dataset dado o bien una cantidad definida por ustedes mismos siempre y cuando se argumente la
razón de forma debida.

In [1]:
# Imports para que el proyecto funcione
# En este caso se decidio utilizar tenserflow

import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


Del dataset de PetImages se cargan 3000 de cada tipo (Gato y Perro) para que el modelo se entrene. A partir de esto hacemos un reacomodamiento, resize y finalmente lo cargamos. De ser una imagen corrupta se descarta.

In [13]:

# Directorio de datos
DATADIR = "PetImages"
CATEGORIES = ["Dog", "Cat"]

# Tamaño de las imágenes
IMG_SIZE = 50

# Cargar imágenes
training_data = []
for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path)[:3000]:  # Tomar solo 3000 imágenes
        try:
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            training_data.append([new_array, class_num])
        except Exception as e:
            pass



### Task 1.2 - Construcción del Modelo

Deberá crear un modelo de Deep Learning orientado a imágenes, como una red que usa capas convolucionales,
para poder resolver este laboratorio. Recuerde que deberá definir la arquitectura de su red, aplicando las diferentes
técnicas vistas en clase (dropouts, funciones de activación, padding, stride, etc). Podrá usar la librería que más le
parezca para completar el laboratorio.


In [14]:

# Seteando los datos a un ancho alto y almacenandolos
data = np.array([i[0] for i in training_data]) / 255.0
labels = np.array([i[1] for i in training_data])
data = np.reshape(data, (data.shape[0], IMG_SIZE, IMG_SIZE, 1))

print("Metricas de desempeño")

# 80 training 20 testing
(train_data, test_data, train_labels, test_labels) = train_test_split(data, labels, test_size=0.2, random_state=42)

# Construccion del modelo
model = Sequential()
model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

# Ejecutar el modelo
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Entrenando
model.fit(train_data, train_labels, epochs=10, validation_data=(test_data, test_labels))

Epoch 1/10
150/150 [==============================] - 10s 59ms/step - loss: 0.6899 - accuracy: 0.5171 - val_loss: 0.6746 - val_accuracy: 0.5539
Epoch 2/10
150/150 [==============================] - 8s 56ms/step - loss: 0.6572 - accuracy: 0.6031 - val_loss: 0.6211 - val_accuracy: 0.6658
Epoch 3/10
150/150 [==============================] - 8s 51ms/step - loss: 0.6261 - accuracy: 0.6509 - val_loss: 0.6016 - val_accuracy: 0.6767
Epoch 4/10
150/150 [==============================] - 8s 54ms/step - loss: 0.5807 - accuracy: 0.7003 - val_loss: 0.5830 - val_accuracy: 0.7009
Epoch 5/10
150/150 [==============================] - 9s 58ms/step - loss: 0.5404 - accuracy: 0.7333 - val_loss: 0.5626 - val_accuracy: 0.7310
Epoch 6/10
150/150 [==============================] - 9s 59ms/step - loss: 0.4982 - accuracy: 0.7552 - val_loss: 0.5702 - val_accuracy: 0.7101
Epoch 7/10
150/150 [==============================] - 9s 63ms/step - loss: 0.4541 - accuracy: 0.7916 - val_loss: 0.5104 - val_accuracy: 0.752

### Task 1.3 - Desempeño del Modelo

Al finalizar el entrenamiento de su modelo, permita que se ingresen nuevas imágenes y permita que el modelo la
clasifique. Además recuerde medir el desempeño de su modelo tanto en entrenamiento como en testing. Para ello,
deberá mostrar las métricas de desempeño de su modelo para las fases dichas, además de la evolución de las
métricas durante las diferentes épocas, cuidando siempre no hacer overfitting sobre el dataset.

In [34]:
# De la prueba 1 a la 3 son gatos
# De la prueba 4 a la 7 son perros
img_path = './TestImages/Prueba1.jpg'

try:
    # Cargando la imagen
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = np.array(img) / 255.0
    img = np.reshape(img, (1, IMG_SIZE, IMG_SIZE, 1))

    # Resultado de probabilidad
    prediction = model.predict(img)

    # Eleccion final
    if prediction[0] < 0.5:
        print("La imagen es un perro")
    else:
        print("La imagen es un gato")
except:
    print("Ruta no valida")


1/1 [==============================] - 0s 24ms/step
La imagen es un gato
